# 自走車1 - 智慧避障自走車

使用超音波sensor偵測前方障礙物距離，來實現閃避障礙物的功能

----


## 系統配置

### ESP12的電機板

- 電機電源（VM）：4.5V～36V，可單獨供電
- 控制電源（VIN）：4.5V～9V（10VMAX），可單獨供電
- 但我們這邊把VM, VIN短路，比較省事
- 其中，D1，D3（A電機）；D2，D4（B電機）

![電機板](image/motor_board.png)

### 供電

- 以行動電源供電即可
- 當然，如果能夠分別供電的話，系統會更加的穩定

### 馬達

- TT減速馬達(48:1)

### 超音波測距模組

- HC-SR04+ : 給MicroPython 記得使用這一款，可以吃3.3V、5V的電，因為MicroPython是3.3V的GPIO

----

## 測試馬達

In [2]:
%serialconnect to --port=com13 --baud=115200

Connecting to --port=com13 --baud=115200 
Ready.


In [1]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> Ready.


In [2]:
# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

In [14]:
def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


### 測試前進、後退、左轉、右轉

In [4]:
# 前進
car_fwd()

In [5]:
# 後退
car_rev()

In [9]:
# 右轉
car_right()

In [11]:
# 左轉
car_left()

In [15]:
# 右轉x2
car_right2()

In [16]:
# 左轉x2
car_left2()

In [17]:
# 停止
car_stop()

----

## 超音波元件

使用 HC-SR04+ 超音波元件測距

- [說明文件](http://eshare.stust.edu.tw/EshareFile/2017_5/2017_5_5c1330b6.pdf)：第一頁有原理說明的圖面。(本地端亦有下載[一份](other/hc-sr04說明文件.pdf)
- 音速 340m/s --> 34000cm/s，因此 1cm的距離要花費29.4us
- 因超音波測距時，聲音走的距離是去回一併計算，因此 (29.4*2)=58.8, 但一般都取58us/cm
- 這邊 ping()函式預設量測最大距離為50cm，因此50*58=2900us，一次量測最久不該等超過3000us (3ms)
- 因timeout (即等超過3000us的時間)代表量測不到，或是障礙物距離很遠，因此回傳一極大值999，代表timeout


In [147]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: Ready.


In [20]:
# 340m/s --> 34000cm/s --> 1cm要29.4us --> 去回，所以除以(29.4*2)=58.8，但手冊是寫58us為1cm
# 如果目標是50cm內的量測距離 --> 50*58=2900us為最長時間
# timeout:回傳999

import time
from machine import Pin

def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

while True:
    distance=ping(trigPin=D7,echoPin=D8)
    print('%s cm' % distance)
    time.sleep(1)

14.1207 cm
17.0862 cm
16.3103 cm
16.3276 cm
17.2241 cm
16.3103 cm
.16.931 cm
14.4828 cm


*** Sending Ctrl-C

14.4828 cm


Traceback (most recent call last):
  File "<stdin>", line 48, in <module>
KeyboardInterrupt: 


----

### 燒入MicroPython板子

- MicroPython一上電會執行 main.py 的程式，因此我們使用magic command "sendtofile" 燒入板子裡


In [18]:
%websocketconnect --password 1234 ws://192.168.43.230:8266

** WebSocket connected **
Password: 
WebREPL connected
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


- 燒錄避障小車程式
- 二輪方向為前進方向

In [19]:
%sendtofile main.py

# 定義
import machine
from machine import Pin
from machine import PWM
import time

# PIN Define:
D0 = 16
D1 = 5  #PWM
D2 = 4  #PWM
D3 = 0  #PWM
D4 = 2  #PWM, #Led on board
D5 = 14 #PWM
D6 = 12 #PWM
D7 = 13 #PWM
D8 = 15 #PWM

#Setup PINS
led = machine.Pin(2, machine.Pin.OUT)
for i in range(5):
    led.value(not led.value())

# for motor sheilf
motor_a1 = machine.Pin(D1, machine.Pin.OUT) #A-, speed
motor_a2 = machine.Pin(D3, machine.Pin.OUT) #A+, dir
motor_b1 = machine.Pin(D2, machine.Pin.OUT) #B-, speed
motor_b2 = machine.Pin(D4, machine.Pin.OUT) #B+, dir
FWD = 1
REV = 0

def car_fwd():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(FWD)

def car_rev():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(REV)

def car_stop():
    motor_a1.value(0)
    motor_a2.value(FWD)
    motor_b1.value(0)
    motor_b2.value(FWD)

def car_right():
    motor_a1.value(0)     #r
    #motor_a2.value(FWD)
    motor_b1.value(1)     #l
    motor_b2.value(FWD)

def car_left():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(0)
    #motor_b2.value(REV)

# 快速右轉
def car_right2():
    motor_a1.value(1)
    motor_a2.value(REV)
    motor_b1.value(1)
    motor_b2.value(FWD)    

# 快速左轉
def car_left2():
    motor_a1.value(1)
    motor_a2.value(FWD)
    motor_b1.value(1)
    motor_b2.value(REV)


def ping(trigPin, echoPin):
    '''
        return: distance (cm)
    '''
    trig=Pin(trigPin, Pin.OUT)
    echo=Pin(echoPin, Pin.IN)
    trig.value(1)
    time.sleep_us(10)
    trig.value(0)
    timeout=False
    tm_start=time.ticks_us() 
    while not echo.value(): #wait for HIGH, 3000us timeout
        if(time.ticks_diff(time.ticks_us(), tm_start)>3000):
            timeout=True
            break
    if timeout: #timeout return 0
        pass
    else: #got HIGH pulse:calculate duration
        tm_start=time.ticks_us()
        tm_delta = 0
        while echo.value(): #wait for LOW
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            if(tm_delta>3000):
                timeout=True
                break
        if timeout:
            pass
        else:
            tm_delta = time.ticks_diff(time.ticks_us(), tm_start)
            duration=tm_delta
    
    if timeout:
        return 999 #cm, for timeout
        
    return duration/58

# go~
car_fwd()

while(True):
    distance=ping(trigPin=D7,echoPin=D8)
    if distance>25:
        car_fwd()
    elif distance>15:
        car_stop()
        time.sleep(0.5)
        car_right()
        time.sleep(0.25)
    elif distance<=10:
        car_stop()
        time.sleep(0.5)
        car_rev()
        time.sleep(0.5)
    time.sleep(0.1)

Sent 129 lines (2699 bytes) to main.py.


- 上面的程式避障右轉0.5秒，大概會轉180度，可以改sleep(0.25)轉90度